In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import math
import backtrader as bt

## Geet data

In [18]:
ticker = ["SPY"]
endDate = pd.to_datetime("now")
startDate = endDate - pd.DateOffset(days = 2520)

In [32]:
stockData = yf.download(ticker, startDate, endDate)
stockData.columns = stockData.columns.droplevel(1)
stockData.columns.name = None
stockData.index = pd.to_datetime(stockData.index).tz_localize(None)

[*********************100%***********************]  1 of 1 completed


In [34]:
stockData

,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,
2018-01-04,243.345963,271.609985,272.160004,270.540009,271.200012,80636400
2018-01-05,244.967606,273.420013,273.559998,271.950012,272.510010,83524000
2018-01-08,245.415634,273.920013,274.100006,272.980011,273.309998,57319200
2018-01-09,245.971115,274.540009,275.250000,274.079987,274.399994,57254000
2018-01-10,245.594788,274.119995,274.420013,272.920013,273.679993,69574300
...,...,...,...,...,...,...
2024-11-21,593.669983,593.669983,595.119995,587.450012,593.400024,46750300
2024-11-22,595.510010,595.510010,596.150024,593.150024,593.659973,38226400
2024-11-25,597.530029,597.530029,600.859985,595.200012,599.520020,42441400


In [36]:
data = bt.feeds.PandasData(dataname=stockData)

## Strategy Implementation - Buy & Hold

In [65]:
class BuyAndHold(bt.Strategy):
    def start(self):
        self.val_start = self.broker.get_cash()
    def nextstart(self):
        size = math.floor((self.broker.get_cash() - 10) / self.data[0])
        self.buy(size = size)
    def stop(self):
        self.roi = (self.broker.get_value() / self.val_start) - 1
        print('_' * 50)
        print('BUY & HOLD')
        print(f'Starting Value: ${self.val_start:,.2f}')
        print(f'ROI: {self.roi * 100.0: 2f}%')
        print(f'Annualised: {100 * ((1+self.roi) ** (365/ (endDate - startDate).days))}')
        print(f'Gross Return: ${self.broker.get_value() - self.val_start}')

In [67]:
cerebro = bt.Cerebro()
cerebro.adddata(data) # Add data feed to Cerebro
cerebro.addstrategy(BuyAndHold) # Add Buy and Hold strategy

0

In [69]:
class FixedCommisionScheme(bt.CommInfoBase):
    params = (
    ('commission', 10), # Fixed commission per trade
    ('stocklike', True), # Treat the asset like a stock
    ('commtype', bt.CommInfoBase.COMM_FIXED) # Fixed commission type
    )
    def _getcommission(self, size, price, pseudoexec):
    # Calculate commission based on fixed amount
        return self.p.commission

In [71]:
import datetime
# Configure broker for the Buy and Hold strategy
broker_args = dict(coc=True) # Enable cash-on-cash calculation
cerebro.broker = bt.brokers.BackBroker(**broker_args)
comminfo = FixedCommisionScheme() # Use the fixed commission scheme
cerebro.broker.addcommissioninfo(comminfo)
cerebro.broker.set_cash(100000) # Set initial cash

In [73]:
cerebro.run()

__________________________________________________
BUY & HOLD
Starting Value: $100,000.00
ROI:  120.406972%
Annualised: 112.12777188581013
Gross Return: $120406.9716796875


In [95]:
cerebro.plot(iplot=False, style='candlestick')

[[<Figure size 640x480 with 4 Axes>]]

## Strategy Implementation - DCA (Dollar Cost Averaging)

In [117]:
import backtrader as bt

class BuyAndHold_More_Fund(bt.Strategy):
    params = dict(
        monthly_cash=1000,  # Amount to add monthly
        monthly_range=[5, 20],  # Random days of the month for investment
    )

    def __init__(self):
        # Initialize strategy variables
        self.order = None
        self.totalcost = 0  # Track total invested amount including commission
        self.cost_wo_bro = 0  # Total cost excluding commissions
        self.units = 0  # Number of units bought
        self.times = 0  # Number of times investments were made

    def log(self, txt, dt=None):
        """Logging function for tracking activity."""
        dt = dt or self.datas[0].datetime.date(0)
        print(f'{dt.isoformat()}, {txt}')

    def start(self):
        """Initialize broker settings."""
        self.broker.set_fundmode(fundmode=True, fundstartval=100.0)
        self.cash_start = self.broker.get_cash()
        self.val_start = 100.0

        # Add a timer for monthly investments
        self.add_timer(
            when=bt.timer.SESSION_START,
            monthdays=[i for i in self.p.monthly_range],
            monthcarry=True,
        )

    def notify_timer(self, timer, when, *args):
        """Add monthly cash and invest."""
        self.broker.add_cash(self.p.monthly_cash)
        target_value = self.broker.get_value() + self.p.monthly_cash - 10
        self.order_target_value(target=target_value)

    def notify_order(self, order):
        """Track order completion and log execution details."""
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f'BUY EXECUTED, Price {order.executed.price:.2f}, Comm {order.executed.comm:.2f}')
                self.units += order.executed.size
                self.totalcost += order.executed.value + order.executed.comm
                self.cost_wo_bro += order.executed.value
                self.times += 1
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')
            self.order = None

    def stop(self):
        """Calculate and display the performance metrics."""
        self.roi = (self.broker.get_value() / self.cash_start) - 1
        self.froi = (self.broker.get_fundvalue() - self.val_start) / self.val_start
        value = self.datas[0].close[0] * self.units + self.broker.get_cash()

        # Ensure `endDate` and `actualStart` are properly defined outside the strategy.
        print('-' * 50)
        print('BUY & BUY MORE')
        #print(f'Time in Market: {(endDate - StartDate).days / 365:.1f} years')
        print(f'#Times: {self.times:.0f}')
        print(f'Value: ${value:,.2f}')
        print(f'Cost: ${self.totalcost:,.2f}')
        print(f'Gross Return: ${value - self.totalcost:,.2f}')
        print(f'Gross %: {(value / self.totalcost - 1) * 100:.2f}%')
        print(f'ROI: {self.roi * 100.0:.2f}%')
        print(f'Fund Value: {self.froi:.2f}%')
        print(f'Annualised: {100 * ((1 + self.froi / 100) ** (365 / ((endDate - startDate).days))):.2f}%')
        print('-' * 50)


In [119]:
# BUY and BUY MORE
cerebro1 = bt.Cerebro()
cerebro1.adddata(data) # Add data feed to Cerebro
cerebro1.addstrategy(BuyAndHold_More_Fund) # Add DCA strategy

0

In [121]:
# Configure broker for the DCA strategy
cerebro1.broker = bt.brokers.BackBroker(**broker_args)
cerebro1.broker.addcommissioninfo(comminfo)
cerebro1.broker.set_cash(1000) # Start with a smaller cash amount

In [123]:
# Run strategies
cerebro1.run()

2018-01-08, BUY EXECUTED, Price 273.42, Comm 10.00
2018-01-23, BUY EXECUTED, Price 282.69, Comm 10.00
2018-02-06, BUY EXECUTED, Price 263.93, Comm 10.00
2018-02-21, BUY EXECUTED, Price 271.40, Comm 10.00
2018-03-06, BUY EXECUTED, Price 272.19, Comm 10.00
2018-03-21, BUY EXECUTED, Price 270.95, Comm 10.00
2018-04-06, BUY EXECUTED, Price 265.64, Comm 10.00
2018-04-23, BUY EXECUTED, Price 266.61, Comm 10.00
2018-05-08, BUY EXECUTED, Price 266.92, Comm 10.00
2018-06-06, BUY EXECUTED, Price 275.10, Comm 10.00
2018-06-21, BUY EXECUTED, Price 275.97, Comm 10.00
2018-07-06, BUY EXECUTED, Price 273.11, Comm 10.00
2018-07-23, BUY EXECUTED, Price 279.68, Comm 10.00
2018-08-07, BUY EXECUTED, Price 284.64, Comm 10.00
2018-09-06, BUY EXECUTED, Price 289.03, Comm 10.00
2018-09-21, BUY EXECUTED, Price 293.58, Comm 10.00
2018-10-08, BUY EXECUTED, Price 287.82, Comm 10.00
2018-10-23, BUY EXECUTED, Price 275.01, Comm 10.00
2018-11-06, BUY EXECUTED, Price 273.39, Comm 10.00
2018-11-21, BUY EXECUTED, Price

In [125]:
cerebro1.plot(iplot=False, style='candlestick') # Visualize DCA

[[<Figure size 640x480 with 4 Axes>]]